# Interactive Transects

This notebook illustrates how to make transects of event results that can be interactively updated by selecting the transect.  


### First set up some things needed in notebook....

In [ ]:
%pylab inline
%matplotlib notebook

In [ ]:
from __future__ import print_function
import sys, os
from ipywidgets import interact
from IPython.display import Image, display

In [ ]:
# local module to read an ascii raster file:
import data_tools

### Read in the topography data and define a function to make a contour plot:

In [ ]:
topo_file = '../geoclaw_output/topo_CC.asc'
topo_data_dict = data_tools.read_asc_file(topo_file, verbose=True)

topo = topo_data_dict['values']

In [ ]:
def plot_topo():
    fig = figure(figsize=(6,6))
    ax = axes()
    topo_clines = arange(0,20,2)
    contour(X,Y,topo,topo_clines,colors='k')
    CClatitude = 41.75  # to rescale longitude
    ax.set_aspect(1. / cos(pi*CClatitude/180.)) 
    ax.ticklabel_format(format='plain',useOffset=False)
    return fig

### Read in depth data

In [ ]:
hmax_file = '../geoclaw_output/hmax_CC.asc'
hmax_data_dict = data_tools.read_asc_file(hmax_file, verbose=False)

hmax = hmax_data_dict['values']
X = hmax_data_dict['X']
Y = hmax_data_dict['Y']

## Define class to allow interactive plotting

The class `DrawLine` allows the reader to select a transect and will then interpolate the solution from the specified event to this transect.

We first define a function `topo_func` that allows us to evaluate the topography at any point `(x,y)`.

The function `plot_transect` is then defined to take two points `(x1,y1)` and `(x2,y2)` and interpolate the topography onto a set of 1000 equally spaced points along the transect (straight line connecting the points).  The function also takes another argument, an `hmax_func` function that can also be evaluated at any point and is assumed to return a value of `hmax` at this point.

In [ ]:
from scipy.interpolate import RegularGridInterpolator
topo_func = RegularGridInterpolator((X[0,:], Y[:,0]), topo.T)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

class DrawLine:
    def __init__(self, fig,ax1,ax2,hmax_func,topo_func):
        self.figure = fig
        self.ax1 = ax1
        self.ax2 = ax2
        self.xprev = 0
        self.yprev = 0
        self.xnew = 0
        self.ynew = 0
        self.press = None
        self.hmax_func = hmax_func
        self.topo_func = topo_func

    def connect(self):
        'connect to all the events we need'
        self.cidpress = self.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)

    def on_press(self, event):
        self.xprev = event.xdata
        self.yprev = event.ydata
        self.press = 1
        ax1.plot(self.xprev,self.yprev,'ro')
        self.figure.canvas.draw()


    def on_release(self, event):
        self.press = None
        self.xnew = event.xdata
        self.ynew = event.ydata
        self.ax1.cla()
        
        topo_clines = arange(0,20,2)
        self.ax1.contour(X,Y,topo,topo_clines,colors='k')
        CClatitude = 41.75  # to rescale longitude
        self.ax1.set_aspect(1. / cos(pi*CClatitude/180.)) 
        self.ax1.ticklabel_format(format='plain',useOffset=False)
    
        self.ax1.plot(self.xprev,self.yprev,'ro')
        self.ax1.plot([self.xprev,self.xnew],[self.yprev,self.ynew],'-r',lw=3)
        
        self.plot_transect()
        self.figure.canvas.draw()

    def disconnect(self):
        'disconnect all the stored connection ids'
        self.figure.canvas.mpl_disconnect(self.cidpress)
        self.figure.canvas.mpl_disconnect(self.cidrelease)
        
    def plot_transect(self):
        # points on transect:
        xi = linspace(self.xprev, self.xnew, 1000)
        yi = linspace(self.yprev, self.ynew, 1000)
        
        # evaulate topo and zeta on transect:
        Bi = self.topo_func(list(zip(xi,yi)))
        zi = self.hmax_func(list(zip(xi,yi)))
                
        # define surface eta as water depth + topography 
        eta = zi+Bi
    
        # Clear axis 2
        self.ax2.cla()
    
        # plot vs. longitude or latitude depending on orientation:
        if (abs(self.xnew-self.xprev) > 0.5*abs(self.ynew-self.yprev)):
            ti = xi
            self.ax2.set_xlim(min(self.xprev,self.xnew),max(self.xprev,self.xnew))
            xtext = 'longitude'
        else:
            ti = yi
            self.ax2.set_xlim(min(self.yprev,self.ynew),max(self.yprev,self.ynew))
            xtext = 'latitude'
            
        BiPos = where(Bi<0, 0., Bi)
        BiNeg = where(Bi>0, 0, Bi)
        
        
        self.ax2.fill_between(ti, BiPos, eta, color='b')       # flood water
        self.ax2.fill_between(ti, BiNeg, 0, color=[.5,.5,1])   # original water
        self.ax2.fill_between(ti, -1e6, Bi, color=[.5,1,.5])   # topography
        self.ax2.plot(ti, Bi, 'g', lw=1)                       # topography
        
        self.ax2.set_xlabel(xtext)
        self.ax2.set_ylabel('meters')
        self.ax2.set_title('Elevation vs. %s' % xtext)
    
        # choose limits of vertical axis to give nice plots:
        eta_wet_max = eta.max()  #where(zi>0, zi+Bi, 0).max()
        y2 = max(10,eta_wet_max)
        self.ax2.set_ylim(-5, y2)
    
        self.ax2.ticklabel_format(format='plain',useOffset=False)
        self.ax1.set_title('(%8.4f,%8.4f) to (%8.4f,%8.4f)' % (self.xprev,self.yprev,self.xnew,self.ynew))

## Select an event and execute to start the interactive plot...

In [ ]:
hmax_func = RegularGridInterpolator((X[0,:], Y[:,0]), hmax.T)

gs = gridspec.GridSpec(10, 3)
fig = plt.figure(figsize=(12,10))
ax1 = plt.subplot(gs[0:6,0:])
ax2 = plt.subplot(gs[7:,0:])

topo_clines = arange(0,20,2)
ax1.contour(X,Y,topo,topo_clines,colors='k')
CClatitude = 41.75  # to rescale longitude
ax1.set_aspect(1. / cos(pi*CClatitude/180.)) 
ax1.ticklabel_format(format='plain',useOffset=False)
extent = (-124.21514, -124.17486, 41.739671,41.762726)   #small region
ax1.axis(extent, aspect=0.1)
ax1.set_xlabel("Longitude")
ax1.set_ylabel("Latitude")
ax1.set_title("Click a point, drag, and release")

dr =  DrawLine(fig,ax1,ax2,hmax_func,topo_func)
dr.connect()

**Note:** To use this, click on a point in the map, drag the mouse, and then release.  The points and transect do not show up until you release.  It would be nice if they did.